In [1]:
import pandas as pd

In [2]:
df_country = pd.read_csv('WDICountry.csv')

In [3]:
df = pd.read_csv('WDICSV.csv')

In [4]:
df_melted = pd.melt(df, id_vars=['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'],
        value_vars=df.columns[4:].to_list(), var_name='Year', value_name='Value')

In [5]:
# Assuming `df` is your DataFrame
# Step 1: Group by 'Indicator Name' and 'Year' and calculate NaN proportions
nan_proportions = df_melted.groupby(['Indicator Name', 'Year'])['Value'].apply(lambda x: x.isna().mean())

# Step 2: Filter out groups where NaN proportion is greater than 50%
valid_groups = nan_proportions[nan_proportions <= 0.5].index

# Step 3: Filter the original DataFrame to keep only the valid groups
df_filtered = df_melted[df_melted.set_index(['Indicator Name', 'Year']).index.isin(valid_groups)]

df_filtered.head(2)

,Country Name,Country Code,Indicator Name,Indicator Code,Year,Value
50,Africa Eastern and Southern,AFE,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1960,140.180526
56,Africa Eastern and Southern,AFE,Age dependency ratio (% of working-age populat...,SP.POP.DPND,1960,89.594604


In [6]:
print(df_filtered.shape)
# Step 1: Group by 'Indicator Name' and 'Year' and calculate zero proportions
zero_proportions = df_filtered.groupby(['Indicator Name', 'Year'])['Value'].apply(lambda x: (x == 0).mean())

# Step 2: Filter out groups where zero proportion is 100%
# valid_groups = zero_proportions[zero_proportions < 1.0].index
valid_groups = zero_proportions[zero_proportions < 0.92].index


# Step 3: Filter the original DataFrame to keep only the valid groups
df_filtered_zeros = df_filtered[df_filtered.set_index(['Indicator Name', 'Year']).index.isin(valid_groups)]
print(df_filtered_zeros.shape)

(8955688, 6)
(8951432, 6)


In [7]:
df_filtered_zeros = df_filtered_zeros.dropna()

In [8]:
dfm = df_filtered_zeros.merge(df_country, on='Country Code')

In [9]:
dfm = dfm[['Country Name',  'Region', 'Indicator Name', 'Year','Value']]

In [10]:
dfm.to_parquet('WDIData1960_to_2024.parquet')

In [ ]:
df = pd.read_parquet('WDIData1960_to_2024.parquet')

In [ ]:
df[df['Indicator Name']== 'Mobile cellular subscriptions (per 100 people)'].sort_values(by='Year').head(100)

In [ ]:
df = df[['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code', '2022']]

In [ ]:
dfm= df.merge(df_country, on='Country Code')#.head()

In [ ]:
dfm = dfm.dropna(subset=['2022', 'Region'])

In [ ]:
dfm = dfm[['Country Name', 'Country Code', 'Region', 'Indicator Name', 'Indicator Code', '2022']]

In [ ]:
dfm = dfm.pivot(index=['Country Name', 'Region'], columns='Indicator Name', values='2022')

In [ ]:
dfm = dfm.dropna(thresh=dfm.shape[0]*0.6,axis=1)

In [ ]:
dfm.to_parquet('WDIData2022reduced.parquet')